# Using a new dataset

In this tutorial we show how you can use a dataset not present in the library.

This particular example uses the ENZIMES dataset, uses a simplicial lifting to create simplicial complexes, and trains the SCN2 model. We train the model using the appropriate training and validation datasets, and finally test it on the test dataset.

### <font color='289C4E'>Table of contents<font><a class='anchor' id='top'></a>
&emsp;[1. Imports](##sec1)

&emsp;[2. Configurations and utilities](##sec2)

&emsp;[3. Loading the data](##sec3)

&emsp;[4. Model initialization](##sec4)

&emsp;[5. Training](##sec5)

&emsp;[6. Testing the model](##sec6)

In [1]:
import hydra
from hydra import compose, initialize
from hydra.utils import instantiate



from topobench.utils.config_resolvers import (
    get_default_metrics,
    get_default_trainer,
    get_default_transform,
    get_flattened_channels,
    get_monitor_metric,
    get_monitor_mode,
    get_non_relational_out_channels,
    get_required_lifting,
    infer_in_channels,
    infer_num_cell_dimensions,
)


with initialize(config_path="../configs", job_name="job"):

    cfg = compose(
        config_name="run.yaml",
        overrides=[
            "model=hypergraph/unignn2",
            "dataset=graph/DAC",
        ], 
        return_hydra_config=True
    )
loader = instantiate(cfg.dataset.loader)


dataset, dataset_dir = loader.load()


C:\Users\acavallo\AppData\Local\Temp\ipykernel_8152\2256932753.py:21: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path="../configs", job_name="job"):
c:\Users\acavallo\AppData\Local\miniconda3\envs\tb\Lib\site-packages\outdated\__init__.py:36: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


Using split number  2


In [2]:
"""Test pipeline for a particular dataset and model."""
import sys 
sys.path.append("../")
sys.path.append("../../")
import hydra
from test._utils.simplified_pipeline import run


DATASET = "graph/DAC"                                                 # ADD YOUR DATASET HERE
MODELS   = ["graph/gcn"]        # ADD ONE OR SEVERAL MODELS OF YOUR CHOICE HERE


class TestPipeline:
    """Test pipeline for a particular dataset and model."""

    def setup_method(self):
        """Setup method."""
        hydra.core.global_hydra.GlobalHydra.instance().clear()
    
    def test_pipeline(self):
        """Test pipeline."""
        with hydra.initialize(config_path="../configs", job_name="job"):
            for MODEL in MODELS:
                cfg = hydra.compose(
                    config_name="run.yaml",
                    overrides=[
                        f"model={MODEL}",
                        f"dataset={DATASET}", # IF YOU IMPLEMENT A LARGE DATASET WITH AN OPTION TO USE A SLICE OF IT, ADD BELOW THE CORRESPONDING OPTION
                        "trainer.max_epochs=2",
                        "trainer.min_epochs=1",
                        "trainer.check_val_every_n_epoch=1",
                        "paths=test",
                        "callbacks=model_checkpoint",
                    ],
                    return_hydra_config=True
                )
                run(cfg)

In [3]:
test = TestPipeline()
test.setup_method()
test.test_pipeline()

C:\Users\acavallo\AppData\Local\Temp\ipykernel_8152\661924356.py:22: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with hydra.initialize(config_path="../configs", job_name="job"):


Using split number  2


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\acavallo\AppData\Local\miniconda3\envs\tb\Lib\site-packages\lightning\pytorch\utilities\parsing.py:44: Attribute 'backbone_wrapper' removed from hparams because it cannot be pickled. You can suppress this warning by setting `self.save_hyperparameters(ignore=['backbone_wrapper'])`.
c:\Users\acavallo\AppData\Local\miniconda3\envs\tb\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:654: Checkpoint directory C:\Users\acavallo\Code\TopoBench\outputs\checkpoints exists and is not empty.

  | Name            | Type                  | Params | Mode 
------------------------------------------------------------------
0 | feature_encoder | AllCellFeatureEncoder | 198    | train
1 | backbone        | GNNWrapper            | 8.4 K  | train
2 | readout         | NoReadOut             | 520    | train
3 | val_acc_best    | MeanMetric            | 0      | train
----

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.
Restoring states from the checkpoint path at C:\Users\acavallo\Code\TopoBench\outputs\checkpoints\epoch_001-v2.ckpt
Loaded model weights from the checkpoint at C:\Users\acavallo\Code\TopoBench\outputs\checkpoints\epoch_001-v2.ckpt
c:\Users\acavallo\AppData\Local\miniconda3\envs\tb\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/accuracy       │    0.11457174271345139    │
│        test/auroc         │     0.482075035572052     │
│         test/loss         │    333.13909912109375     │
│      test/precision       │   0.015568318776786327    │
│        test/recall        │    0.1149553582072258     │
└───────────────────────────┴───────────────────────────┘

## 1. Imports <a class="anchor" id="sec1"></a>

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import lightning as pl
# Hydra related imports
from omegaconf import OmegaConf
# Dataset related imports
from torch_geometric.datasets import TUDataset
# from topobench.dataloader.dataloader import TBDataloader
from topobench.data.loaders.graph.dac_dataset_loader import DACDatasetLoader
from topobench.data.preprocessor import PreProcessor
# Model related imports
from topobench.model.model import TBModel
from topomodelx.nn.simplicial.scn2 import SCN2
from topobench.nn.wrappers.simplicial import SCNWrapper
from topobench.nn.encoders import AllCellFeatureEncoder
from topobench.nn.readouts import PropagateSignalDown
# Optimization related imports
from topobench.loss.loss import TBLoss
from topobench.optimizer import TBOptimizer
from topobench.evaluator.evaluator import TBEvaluator

c:\Users\acavallo\AppData\Local\miniconda3\envs\tb\Lib\site-packages\outdated\__init__.py:36: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


In [2]:
from topobench.data.datasets import DACDataset

data = DACDataset(root='./data/4_125/', name='4_125')

TypeError: DACDataset.__init__() missing 1 required positional argument: 'parameters'

## 2. Configurations and utilities <a class="anchor" id="sec2"></a>

Configurations can be specified using yaml files or directly specified in your code like in this example.

In [2]:
transform_config = { "clique_lifting":
    {"transform_type": "lifting",
    "transform_name": "SimplicialCliqueLifting",
    "complex_dim": 3,}
}

split_config = {
    "learning_setting": "inductive",
    "split_type": "random",
    "data_seed": 0,
    "data_split_dir": "./data/ENZYMES/splits/",
    "train_prop": 0.5,
}

in_channels = 3
out_channels = 6
dim_hidden = 16

wrapper_config = {
    "out_channels": dim_hidden,
    "num_cell_dimensions": 3,
}

readout_config = {
    "readout_name": "PropagateSignalDown",
    "num_cell_dimensions": 1,
    "hidden_dim": dim_hidden,
    "out_channels": out_channels,
    "task_level": "graph",
    "pooling_type": "sum",
}

loss_config = {
    "dataset_loss": 
        {
            "task": "classification", 
            "loss_type": "cross_entropy"
        }
}

evaluator_config = {"task": "classification",
                    "num_classes": out_channels,
                    "metrics": ["accuracy", "precision", "recall"]}

optimizer_config = {"optimizer_id": "Adam",
                    "parameters":
                        {"lr": 0.001,"weight_decay": 0.0005}
                    }

transform_config = OmegaConf.create(transform_config)
split_config = OmegaConf.create(split_config)
readout_config = OmegaConf.create(readout_config)
loss_config = OmegaConf.create(loss_config)
evaluator_config = OmegaConf.create(evaluator_config)
optimizer_config = OmegaConf.create(optimizer_config)

In [3]:
def wrapper(**factory_kwargs):
    def factory(backbone):
        return SCNWrapper(backbone, **factory_kwargs)
    return factory

## 3. Loading the data <a class="anchor" id="sec3"></a>

In this example we use the ENZYMES dataset. It is a graph dataset and we use the clique lifting to transform the graphs into simplicial complexes. We invite you to check out the README of the [repository](https://github.com/pyt-team/TopoBenchX) to learn more about the various liftings offered.

In [4]:
dataset_dir = "./data/ENZYMES/"
dataset = TUDataset(root=dataset_dir, name="ENZYMES")

preprocessor = PreProcessor(dataset, dataset_dir, transform_config)
dataset_train, dataset_val, dataset_test = preprocessor.load_dataset_splits(split_config)
datamodule = TBDataloader(dataset_train, dataset_val, dataset_test, batch_size=32)

Transform parameters are the same, using existing data_dir: ./data/ENZYMES/clique_lifting/3206123057


## 4. Model initialization <a class="anchor" id="sec4"></a>

We can create the backbone by instantiating the SCN2 model from TopoModelX. Then the `SCNWrapper` and the `TBModel` take care of the rest.

In [5]:
backbone = SCN2(in_channels_0=dim_hidden, in_channels_1=dim_hidden, in_channels_2=dim_hidden)
wrapper = wrapper(**wrapper_config)

readout = PropagateSignalDown(**readout_config)
loss = TBLoss(**loss_config)
feature_encoder = AllCellFeatureEncoder(in_channels=[in_channels, in_channels, in_channels], out_channels=dim_hidden)

evaluator = TBEvaluator(**evaluator_config)
optimizer = TBOptimizer(**optimizer_config)

In [6]:
model = TBModel(backbone=backbone,
                 backbone_wrapper=wrapper,
                 readout=readout,
                 loss=loss,
                 feature_encoder=feature_encoder,
                 evaluator=evaluator,
                 optimizer=optimizer,
                 compile=False,)

## 5. Training <a class="anchor" id="sec5"></a>

Now we can use the `lightning` trainer to train the model.

In [7]:
#%%capture
# Increase the number of epochs to get better results
trainer = pl.Trainer(max_epochs=5, accelerator="cpu", enable_progress_bar=False)

trainer.fit(model, datamodule)
train_metrics = trainer.callback_metrics

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/levtel/miniconda3/envs/topobench/lib/python3.11/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/home/levtel/miniconda3/envs/topobench/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/home/levtel/miniconda3/envs/topobench/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:44: Attribute 'backbone_wrapper' r

/home/levtel/miniconda3/envs/topobench/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.
/home/levtel/miniconda3/envs/topobench/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: The ``compute`` method of metric MulticlassAccuracy was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)
/home/levtel/miniconda3/envs/topobench/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: The ``compute`` method of metric MulticlassPrecision was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)
/home/levtel/miniconda3/envs/topobench/lib/p

In [8]:
print('      Training metrics\n', '-'*26)
for key in train_metrics:
    print('{:<21s} {:>5.4f}'.format(key+':', train_metrics[key].item()))

      Training metrics
 --------------------------
train/accuracy:       0.1867
train/precision:      0.1796
train/recall:         0.1835
val/loss:             3.2280
val/accuracy:         0.1600
val/precision:        0.1735
val/recall:           0.1554
train/loss:           3.2763


## 6. Testing the model <a class="anchor" id="sec6"></a>

Finally, we can test the model and obtain the results.

In [9]:
trainer.test(model, datamodule)
test_metrics = trainer.callback_metrics

/home/levtel/miniconda3/envs/topobench/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/accuracy       │    0.18000000715255737    │
│         test/loss         │     3.625048875808716     │
│      test/precision       │    0.1994038224220276     │
│        test/recall        │    0.1821674406528473     │
└───────────────────────────┴───────────────────────────┘

In [10]:
print('      Testing metrics\n', '-'*25)
for key in test_metrics:
    print('{:<20s} {:>5.4f}'.format(key+':', test_metrics[key].item()))

      Testing metrics
 -------------------------
test/loss:           3.6250
test/accuracy:       0.1800
test/precision:      0.1994
test/recall:         0.1822
